In [1]:
import os
import time
import numpy as np
import pandas as pd
import scipy.sparse as sparse
import itertools

In [2]:
!pip install -qU kaggle

In [3]:
try:
    import kaggle
except OSError as e:
    print(e)

Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
kaggle_dir = '/root/.kaggle/'
os.makedirs(kaggle_dir, exist_ok=True)

In [7]:
import json

kaggle_credentials = {"username": "tanbinislamrohan", "key": "9d69966a675f34f574f9d10b73cc963b"}
kaggle_file_path = os.path.join(kaggle_dir, 'kaggle.json')
with open(kaggle_file_path, 'w') as fp:
    json.dump(kaggle_credentials, fp)

In [ ]:
!kaggle competitions download -c instacart-market-basket-analysis -p '/content/drive/My Drive/'


In [10]:
import zipfile

files = [
    '/content/drive/My Drive/instacart-market-basket-analysis.zip',
    '/content/drive/My Drive/order_products__train.csv.zip',
    '/content/drive/My Drive/order_products__prior.csv.zip',
    '/content/drive/My Drive/products.csv.zip',
    '/content/drive/My Drive/orders.csv.zip'
]

for file_path in files:
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall('/content/drive/My Drive/')

In [11]:
order_products_train = pd.read_csv('/content/drive/My Drive/order_products__train.csv')
order_products_prior = pd.read_csv('/content/drive/My Drive/order_products__prior.csv')
products = pd.read_csv('/content/drive/My Drive/products.csv')
orders = pd.read_csv('/content/drive/My Drive/orders.csv')

order_products = pd.concat([order_products_train, order_products_prior])

In [12]:
customer_order_products = pd.merge(orders, order_products, how='inner',on='order_id')

# creating a table with "confidences"
data = customer_order_products.groupby(['user_id', 'product_id'])[['order_id']].count().reset_index()
data.columns=["user_id", "product_id", "total_orders"]
data.product_id = data.product_id.astype('int64')

# Create a lookup frame so we can get the product names back in readable form later.
products_lookup = products[['product_id', 'product_name']].drop_duplicates()
products_lookup['product_id'] = products_lookup.product_id.astype('int64')

In [13]:
data_new = pd.DataFrame([[data.user_id.max() + 1, 22802, 97],
                         [data.user_id.max() + 2, 26834, 89],
                         [data.user_id.max() + 2, 12590, 77]
                        ], columns=['user_id', 'product_id', 'total_orders'])

In [14]:
users = list(np.sort(data.user_id.unique()))
items = list(np.sort(products.product_id.unique()))
purchases = list(data.total_orders)

# create zero-based index position <-> user/item ID mappings
index_to_user = pd.Series(users)

# create reverse mappings from user/item ID to index positions
user_to_index = pd.Series(data=index_to_user.index + 1, index=index_to_user.values)

# create zero-based index position <-> item/user ID mappings
index_to_item = pd.Series(items)

# create reverse mapping from item/user ID to index positions
item_to_index = pd.Series(data=index_to_item.index, index=index_to_item.values)

# Get the rows and columns for our new matrix
products_rows = data.product_id.astype(int)
users_cols = data.user_id.astype(int)

# Create a sparse matrix for our users and products containing number of purchases
sparse_product_user = sparse.csr_matrix((purchases, (products_rows, users_cols)), shape=(len(items) + 1, len(users) + 1))
sparse_product_user.data = np.nan_to_num(sparse_product_user.data, copy=False)

sparse_user_product = sparse.csr_matrix((purchases, (users_cols, products_rows)), shape=(len(users) + 1, len(items) + 1))
sparse_user_product.data = np.nan_to_num(sparse_user_product.data, copy=False)

In [15]:
!pip install -qU implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 62.8 MB/s eta 0:00:00


In [16]:
import implicit
from implicit import evaluation

#split data into train and test sets
train_set, test_set = evaluation.train_test_split(sparse_user_product, train_percentage=0.9)

# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=100,
                                             regularization=0.05,
                                             iterations=50,
                                             num_threads=1)

alpha_val = 15
train_set = (train_set * alpha_val).astype('double')

# train the model on a sparse matrix of item/user/confidence weights
model.fit(train_set, show_progress = True)

  0%|          | 0/50 [00:00<?, ?it/s]

In [17]:
test_set = (test_set * alpha_val).astype('double')
evaluation.ranking_metrics_at_k(model, train_set, test_set, K=100,
                         show_progress=True, num_threads=1)

  0%|          | 0/192840 [00:00<?, ?it/s]

{'precision': 0.27559960803046857,
 'map': 0.04430799689100116,
 'ndcg': 0.1440653652536139,
 'auc': 0.6551755461580111}

In [18]:
!pip install -qU pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 20.5 MB/s eta 0:00:00


In [19]:
import pinecone
api_key = os.getenv('PINECONE_API_KEY') or '3d2dafd8-6156-4d84-b9c1-bc81e228f9f3'
env = os.getenv('PINECONE_ENVIRONMENT') or "gcp-starter"

pinecone.init(api_key=api_key, environment=env)

In [20]:
index_name = 'product-recommender'

In [21]:
if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)
pinecone.create_index(name=index_name, dimension=100)

In [22]:
index = pinecone.Index(index_name=index_name)

In [23]:
import torch
all_items_titles = [{'title': title} for title in products_lookup['product_name']]
all_items_ids = [str(product_id) for product_id in products_lookup['product_id']]
items_factors = model.item_factors
device = "cuda" if torch.cuda.is_available() else "cpu"
item_embeddings = items_factors[1:].to_numpy().tolist() if device == "cuda" else items_factors[1:].tolist()
items_to_insert = list(zip(all_items_ids, item_embeddings, all_items_titles))
display(items_to_insert[:2])

[('1',
  [-0.015056516043841839,
   0.00021836196538060904,
   0.0023376436438411474,
   0.005412748549133539,
   0.007761726621538401,
   -0.0012462955201044679,
   -0.013322396203875542,
   -0.008758085779845715,
   -0.006212309468537569,
   0.005777330137789249,
   -0.0051048225723207,
   0.014401901513338089,
   0.01158350519835949,
   -0.013323500752449036,
   -0.005051496904343367,
   0.009174949489533901,
   -0.0007089755963534117,
   -0.0022933233994990587,
   -0.016390731558203697,
   -0.023660454899072647,
   -0.0014584538294002414,
   0.002086278982460499,
   0.0008372272714041173,
   0.008204532787203789,
   0.01367188896983862,
   -0.004961831495165825,
   0.021037157624959946,
   0.016908757388591766,
   0.014376295730471611,
   0.0016403045738115907,
   0.017297083511948586,
   -0.010524523444473743,
   -0.013444743119180202,
   0.012396245263516903,
   -0.0004208516329526901,
   -0.009498516097664833,
   0.008551465347409248,
   -0.015620090067386627,
   -0.003798287129

In [24]:
from tqdm.auto import tqdm

BATCH_SIZE = 100

print('Index statistics before upsert:', index.describe_index_stats())

for i in tqdm(range(0, len(items_to_insert), BATCH_SIZE)):
        index.upsert(vectors=items_to_insert[i:i+BATCH_SIZE])

print('Index statistics after upsert:', index.describe_index_stats())

Index statistics before upsert: {'dimension': 100,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


  0%|          | 0/497 [00:00<?, ?it/s]

Index statistics after upsert: {'dimension': 100,
 'index_fullness': 0.452,
 'namespaces': {'': {'vector_count': 45200}},
 'total_vector_count': 45200}


In [25]:
def products_bought_by_user_in_the_past(user_id: int, top: int = 10):

    selected = data[data.user_id == user_id].sort_values(by=['total_orders'], ascending=False)

    selected['product_name'] = selected['product_id'].map(products_lookup.set_index('product_id')['product_name'])
    selected = selected[['product_id', 'product_name', 'total_orders']].reset_index(drop=True)
    if selected.shape[0] < top:
        return selected

    return selected[:top]

In [28]:
data.head()

,user_id,product_id,total_orders
0,1,196,11
1,1,10258,10
2,1,10326,1
3,1,12427,10
4,1,13032,4


In [29]:
user_ids = [10258, 10326, 12427]
user_factors = model.user_factors[user_to_index[user_ids]]

display(user_factors[1:])

Matrix([[ 0.9877756   0.45237714 -1.0820059   1.1346567  -0.33615407  1.2228149
  -0.3470497  -0.21542554 -1.4627649  -0.5249924  -0.56351614  0.48190752
  -0.8522596   0.37146875  1.722827    1.1394941   0.31025606 -0.03543369
  -0.41101316  1.0162079   0.5464323   1.336701   -0.15127829 -0.19205385
   0.83167654 -0.19290085 -0.07175911  0.49425584  1.4051131   1.2001904
  -0.05038859 -0.9829761  -0.13594334  0.7947231  -0.32624087 -0.8158287
  -0.1848513  -1.0921514   0.5549813   1.2016124  -0.62300086 -0.03818626
  -0.72533286  0.9455901  -0.70663947 -1.6313971  -1.0059937  -0.4427504
   1.1844205  -1.626552    0.7679522   0.30196324 -0.54507697 -0.8358914
   0.9333731  -1.0153251  -0.33208543 -0.764874   -0.86192775 -0.8602954
  -0.9544375   0.3190677  -0.27641955 -0.31062898 -0.5546651  -0.5118046
  -0.14061056  0.57039845 -1.4331974  -0.5220096   0.6270259  -0.21750206
   0.6887823  -0.37778986  0.60196304  0.7716483  -1.3171916  -0.03289795
  -0.08535727 -0.6745469   0.957463   

In [30]:
print("Model recommendations\n")

start_time = time.process_time()
recommendations0 = model.recommend(userid=user_ids[0], user_items=sparse_user_product[0])
recommendations1 = model.recommend(userid=user_ids[1], user_items=sparse_user_product[1])
recommendations2 = model.recommend(userid=user_ids[2], user_items=sparse_user_product[2])
print("Time needed for retrieving recommended products: " + str(time.process_time() - start_time) + ' seconds.\n')

print('\nRecommendations for person 0:')
for recommendation in recommendations0[0]:
    print(products_lookup[products_lookup.product_id == recommendation]['product_name'].values)

print('\nRecommendations for person 1:')
for recommendation in recommendations1[0]:
    print(products_lookup[products_lookup.product_id == recommendation]['product_name'].values)

print('\nRecommendations for person 2:')
for recommendation in recommendations2[0]:
    print(products_lookup[products_lookup.product_id == recommendation]['product_name'].values)

Model recommendations

Time needed for retrieving recommended products: 0.004193247999992877 seconds.


Recommendations for person 0:
['Organic Lacinato (Dinosaur) Kale']
['Organic Cilantro']
['Organic Coconut Milk']
['Organic Italian Parsley Bunch']
['Organic Garlic']
['Organic Baby Arugula']
['Carrots']
['Fresh Cauliflower']
['Organic Small Bunch Celery']
['Organic Yellow Onion']

Recommendations for person 1:
['Vanilla Almond Breeze Almond Milk']
['Organic Baby Arugula']
['Organic Basil']
['Organic Hass Avocado']
['Extra Virgin Olive Oil']
['Limes']
['Organic Garlic']
['Asparagus']
['Organic Avocado']
['100% Raw Coconut Water']

Recommendations for person 2:
['Organic Fuji Apple']
['Frozen Broccoli Florets']
['Cucumber Kirby']
['Pineapple Chunks']
['Organic Baby Carrots']
['Michigan Organic Kale']
['Seedless Red Grapes']
['Blueberries']
['Chopped Spinach']
['Berry Medley']


In [ ]:
user_embeddings = user_factors.to_numpy()[:-1].tolist() if device == "cuda" else user_factors[:-1].tolist()

start_time = time.process_time()
query_results = index.query(
    vector=user_embeddings,
    filter={
        "genre": {"$eq": "documentary"},
        "year": 2019
    },
    top_k=10,
    include_metadata=True
)
print("Time needed for retrieving recommended products using Pinecone: " + str(time.process_time() - start_time) + ' seconds.\n')

for _id, res in zip(user_ids, query_results.results):
    print(f'user_id={_id}')
    df = pd.DataFrame(
        {
            'products': [match.metadata['title'] for match in res.matches],
            'scores': [match.score for match in res.matches]
        }
    )
    print("Recommendation: ")
    display(df)
    print("Top buys from the past: ")
    display(products_bought_by_user_in_the_past(_id, top=15))